# NN Model

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error as mse 
# local imports
from model_tools import PredictorData
from nn_models import TF_NN_Predictor
from utils import Utils

2024-06-09 17:19:49.983843: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-09 17:19:50.148567: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-09 17:19:50.986744: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-09 17:19:56.142755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = Utils.read_config_for_env(config_path='../config/config.yml')
pred_data = PredictorData(
    config,
    refresh_monthly=False,
    refresh_ts_features=False,
    clean_strategy='olrem_for_all',
    # split_strategy='random',
    # split_strategy='last_months_val',
    split_strategy='months',
    num_lag_mon=3,
    val_ratio=0.2)

Loading data.. Done.
Fixing data schemas.. Done.
Loading /home/onur/WORK/DS/repos/shop_sales_prediction/data/all_ts.parquet


## NN model with 3 Dense layers

In [3]:
tf_nn_predictor_3D = TF_NN_Predictor(
    pred_data=pred_data,
    output_dim=1,
    model_name='3Dense',
    optimizer="adam",
    metrics=['mse'])

In [4]:
tf_nn_predictor_3D.pred_data = tf_nn_predictor_3D.split_transform(
    tf_nn_predictor_3D.pred_data,
    tf_nn_predictor_3D.transformer)

Splitting train-val
Fit-transforming X_train
Transforming X_val


In [5]:
tf_nn_predictor_3D.create_model()

Creating model


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,753 (10.75 KB)

 Trainable params: 2,753 (10.75 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
#Train the model
tf_nn_predictor_3D.model.fit(
    tf_nn_predictor_3D.pred_data.X_train,
    tf_nn_predictor_3D.pred_data.y_train,
    epochs=10,
    batch_size=4096,
    validation_data=(tf_nn_predictor_3D.pred_data.X_val, tf_nn_predictor_3D.pred_data.y_val),
    callbacks=tf_nn_predictor_3D.callbacks)

Epoch 1/10


2024-06-09 17:21:14.544048: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2490134400 exceeds 10% of free system memory.


8443/8444 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3454 - mse: 0.3454
Epoch 1: val_loss improved from inf to 0.43440, saving model to 3Dense_weights-001--0.43440.keras
8444/8444 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - loss: 0.3454 - mse: 0.3454 - val_loss: 0.4344 - val_mse: 0.4344
Epoch 2/10
8441/8444 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2974 - mse: 0.2974
Epoch 2: val_loss did not improve from 0.43440
8444/8444 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - loss: 0.2974 - mse: 0.2974 - val_loss: 0.4501 - val_mse: 0.4501
Epoch 3/10
8439/8444 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2975 - mse: 0.2975
Epoch 3: val_loss improved from 0.43440 to 0.42733, saving model to 3Dense_weights-003--0.42733.keras
8444/8444 ━━━━━━━━━━━━━━━━━━━━ 88s 10ms/step - loss: 0.2975 - mse: 0.2975 - val_loss: 0.4273 - val_mse: 0.4273
Epoch 4/10
8441/8444 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2949 - mse: 0.2949
Epoch 4: val_loss did not improve from 0.42733
8444/8444 ━━━━━━━━━━━━━━━━━━━━ 104s 12ms/step - loss:

In [7]:
# Make predictions on the training and validation data. 
pred_train = tf_nn_predictor_3D.model.predict_on_batch(pred_data.X_train)
pred_val = tf_nn_predictor_3D.model.predict_on_batch(pred_data.X_val)

# Calculate and print the Root Mean Squared Error (RMSE) for training and validation predictions. 
print("Training RMSE: ", np.sqrt(mse(pred_data.y_train, pred_train)))
print("Validation RMSE: ", np.sqrt(mse(pred_data.y_val, pred_val)))

2024-06-09 17:29:17.029846: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4980268800 exceeds 10% of free system memory.
2024-06-09 17:29:21.217590: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2490134400 exceeds 10% of free system memory.
2024-06-09 17:29:22.456293: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4426905600 exceeds 10% of free system memory.
2024-06-09 17:29:27.611354: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4426905600 exceeds 10% of free system memory.


Training RMSE:  0.53407484
Validation RMSE:  0.6537046
